# 一.清洗与合并数据

In [22]:
import pandas as pd
import glob
import os
import numpy as np

In [14]:
# 定义数据文件夹的路径
data_path = '../Data/'

# 使用glob库找到所有.xlsx文件
all_files = glob.glob(os.path.join(data_path, "*.xlsx"))

# 打印一下，确认所有文件都已找到
print("找到以下数据文件:")
for f in all_files:
    print(f"- {os.path.basename(f)}")

找到以下数据文件:
- Tianjin_Zhongxin_esf_data.xlsx
- Tianjin_Wuqing_esf_data.xlsx
- Tianjin_Balitai_esf_ data.xlsx
- Tianjin_Quanyechang_esf_data.xlsx
- Tianjin_Quanyechang_zu_data.xlsx
- Tianjin_Wuqing_zu_data.xlsx
- Tianjin_Balitai_zu_data.xlsx
- Tianjin_Zhongxin_zu_data.xlsx


In [29]:
# 3. 创建一个用于存放每个读取到的DataFrame的列表
df_list = []

# 循环遍历文件列表，读取数据并添加标识列
for file_path in all_files:
    # 读取当前的Excel文件
    df = pd.read_excel(file_path)
    
    # 从文件名中提取“地区”和“类型”信息
    file_name = os.path.basename(file_path)
    
    # 清理文件名并按"_"分割
    parts = file_name.replace('.xlsx', '').split('_')
    
    # 提取地区和类型
    location = parts[1] 
    data_type = parts[2]

    # 为DataFrame添加新的列
    df['地区'] = location
    df['类型'] = data_type
    
    # 将处理好的DataFrame添加到列表中
    df_list.append(df)

full_df = pd.concat(df_list, ignore_index=True)
full_df['租金单价(元/㎡/月)'] = np.where(
        full_df['类型'] == 'zu',
        full_df['租金(元/月)'] / full_df['面积(㎡)'],
        np.nan)

full_df

,面积(㎡),总价(万),单价(元/㎡),地区,类型,租金(元/月),租金单价(元/㎡/月)
0,275.10,450.0,16357.0,Zhongxin,esf,NaN,NaN
1,87.00,60.0,6896.0,Zhongxin,esf,NaN,NaN
2,132.64,370.0,27895.0,Zhongxin,esf,NaN,NaN
3,275.00,370.0,13454.0,Zhongxin,esf,NaN,NaN
4,247.00,557.0,22550.0,Zhongxin,esf,NaN,NaN
...,...,...,...,...,...,...,...
8976,83.00,NaN,NaN,Zhongxin,zu,2600.0,31.325301
8977,95.00,NaN,NaN,Zhongxin,zu,2400.0,25.263158
8978,104.00,NaN,NaN,Zhongxin,zu,2700.0,25.961538
8979,93.00,NaN,NaN,Zhongxin,zu,2000.0,21.505376


In [30]:
output_filename = 'cleaned_full_data.parquet'
output_path = os.path.join(data_path, output_filename)
full_df.to_parquet(output_path, index=False, engine='pyarrow')